In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/Jurkat/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/Jurkat/1.Genome_wide_prediction/Signac/Signac.250321/"
celltype = "Jurkat"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [6]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [7]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, 

Prepare background data

In [8]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 152887 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1   10001055-10001570      *
       [2]     chr1 100037799-100039086      *
       [3]     chr1 100046162-100046627      *
       [4]     chr1 100048508-100049104      *
       [5]     chr1 100064686-100065482      *
       ...      ...                 ...    ...
  [152883]     chrX   99271630-99271935      *
  [152884]     chrX     9941883-9942743      *
  [152885]     chrX   99603024-99603776      *
  [152886]     chrX   99632013-99632219      *
  [152887]     chrX     9995829-9996207      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [9]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-10001055-10001570,33,25,37,22,52,32,6,49,19,41,⋯,29,12,41,47,39,50.38760,516,chr1,35,0.28396136
chr1-100037799-100039086,56,50,77,34,61,146,125,92,69,143,⋯,61,31,85,71,49,64.75155,1288,chr1,1750,0.97656439
chr1-100046162-100046627,23,15,30,18,23,33,9,41,24,27,⋯,30,16,32,42,75,46.13734,466,chr1,31,0.19075526
chr1-100048508-100049104,41,30,26,40,38,40,4,70,25,24,⋯,27,32,58,47,76,41.37353,597,chr1,28,0.10588866
chr1-100064686-100065482,50,27,51,52,53,61,3,75,40,42,⋯,41,37,62,69,104,43.16186,797,chr1,38,0.34044098
chr1-10010-10640,84,84,11,1,15,184,20,83,11,22,⋯,9,70,12,11,2,56.41838,631,chr1,218,0.85151779
chr1-100132417-100133466,55,52,67,33,47,96,58,84,70,72,⋯,56,34,65,73,77,56.47619,1050,chr1,1315,0.96289416
chr1-1001752-1002084,6,4,19,3,7,62,39,26,17,47,⋯,14,2,22,20,1,76.87688,333,chr1,286,0.87951886
chr1-100183399-100183907,15,21,38,25,42,41,4,48,16,40,⋯,34,25,34,48,47,50.29470,509,chr1,26,0.05274484


Run Signac prediction

In [10]:
Sys.setenv(OPENBLAS_NUM_THREADS=1)
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 10141211 ranges and 5 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10001055-10001570      * |     AADACL4
         [2]     chr1 10001055-10001570      * |       ACOT7
         [3]     chr1 10001055-10001570      * |      AGTRAP
         [4]     chr1 10001055-10001570      * | APITD1-CORT
         [5]     chr1 10001055-10001570      * |    C1orf158
         ...      ...               ...    ... .         ...
  [10141207]     chrX   9995829-9996207      * |      TCEANC
  [10141208]     chrX   9995829-9996207      * |        TLR7
  [10141209]     chrX   9995829-9996207      * |      TMSB4X
  [10141210]     chrX   9995829-9996207      * |     TRAPPC2
  [10141211]     chrX   9995829-9996207      * |        WWC3
                           PeakName               PairName    zscore    pvalue
                        <character>            <character> <logical>

In [12]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna.rename,
                                 matrix.atac,
                                 meta.features,
                                 8)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 14.78642 hours

In [13]:
execution_time

Time difference of 14.78642 hours

Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 10141211 ranges and 5 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10001055-10001570      * |     AADACL4
         [2]     chr1 10032464-10033474      * |     AADACL4
         [3]     chr1 10045901-10046290      * |     AADACL4
         [4]     chr1 10056450-10057024      * |     AADACL4
         [5]     chr1 10059603-10060323      * |     AADACL4
         ...      ...               ...    ... .         ...
  [10141207]     chrX 97864825-97864975      * |      NAP1L3
  [10141208]     chrX 97866845-97868084      * |      NAP1L3
  [10141209]     chrX 97880136-97880451      * |      NAP1L3
  [10141210]     chrX 98225046-98225768      * |      NAP1L3
  [10141211]     chrX 98415325-98415609      * |      NAP1L3
                           PeakName               PairName    zscore
                        <character>            <character> <numeric>
         

In [15]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,10001055,10001570,516,*,AADACL4,chr1-10001055-10001570,chr1-10001055-10001570_AADACL4,-0.3539336,6.383057e-01,Jurkat
chr1,10032464,10033474,1011,*,AADACL4,chr1-10032464-10033474,chr1-10032464-10033474_AADACL4,4.3663645,6.316580e-06,Jurkat
chr1,10045901,10046290,390,*,AADACL4,chr1-10045901-10046290,chr1-10045901-10046290_AADACL4,1.1409592,1.269435e-01,Jurkat
chr1,10056450,10057024,575,*,AADACL4,chr1-10056450-10057024,chr1-10056450-10057024_AADACL4,0.5986920,2.746891e-01,Jurkat
chr1,10059603,10060323,721,*,AADACL4,chr1-10059603-10060323,chr1-10059603-10060323_AADACL4,2.5099684,6.037098e-03,Jurkat
chr1,10136314,10136779,466,*,AADACL4,chr1-10136314-10136779,chr1-10136314-10136779_AADACL4,-1.1137851,8.673143e-01,Jurkat
chr1,10140677,10141818,1142,*,AADACL4,chr1-10140677-10141818,chr1-10140677-10141818_AADACL4,0.2135944,4.154317e-01,Jurkat
chr1,10152544,10153653,1110,*,AADACL4,chr1-10152544-10153653,chr1-10152544-10153653_AADACL4,-0.5412238,7.058233e-01,Jurkat
chr1,10164960,10165566,607,*,AADACL4,chr1-10164960-10165566,chr1-10164960-10165566_AADACL4,-1.4118368,9.210010e-01,Jurkat


In [16]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2                 dplyr_1.1.4                      
 [3] foreach_1.5.2                     Matrix_1.6